In [1]:
import pandas as pd
import requests
import re

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import os
from dotenv import load_dotenv 

Extraer detalles como:

Artista -> de mis artistas  de spotify -> traer CSV??? 🚨🚨

País de Origen.

Area de Origen.

Fechas inicio de actividad.

Fechas fin de actividad (en caso que tenga o simplemente "actualidad").

Llamada con Endpoint en funcion de los artistas de spotify

Ejemplo de endpoint: https://musicbrainz.org/ws/2/release/?query=album:Abbey%20Road&fmt=json

In [22]:
# Llamada a la API

url = "https://musicbrainz.org/ws/2/artist/?query=artist:shakira&fmt=json"


# Hemos sacado key pero de momento no nos hace falta para la llamada
# api_key = "RjTkSzMRh0RiiMO9oyJicZYIuNPLYVkhVbSRtjDB"
# headers = {"Authorization": api_key}



In [23]:
def llamar_api(url):

    llamada = requests.get(url)  # Realiza una solicitud GET a la URL proporcionada y almacena la respuesta en 'llamada'.
    print(f"La llamada a la API nos ha dado una respuesta de tipo: {llamada.status_code}")  # Imprime el código de estado de la respuesta.
    if llamada.status_code != 200:  # Comprueba si la respuesta no fue exitosa (código de estado distinto de 200).
        print(f"El motivo por el que la llamada falló es {llamada.reason}")  # Imprime la razón de la falla.
    else:
        return llamada.json()  # Si la llamada fue exitosa, devuelve los datos de respuesta en formato JSON.

In [24]:
result = llamar_api(url)
result

La llamada a la API nos ha dado una respuesta de tipo: 200


{'created': '2024-10-23T10:24:04.422Z',
 'count': 13,
 'offset': 0,
 'artists': [{'id': 'bf24ca37-25f4-4e34-9aec-460b94364cfc',
   'type': 'Person',
   'type-id': 'b6e035f4-3ce9-331c-97df-83397230b0df',
   'score': 100,
   'gender-id': '93452b5a-a947-30c8-934f-6a4056b151c2',
   'name': 'Shakira',
   'sort-name': 'Shakira',
   'gender': 'female',
   'country': 'CO',
   'area': {'id': '02b60d8d-7164-339d-868d-22d147d9f74a',
    'type': 'Country',
    'type-id': '06dd0ae4-8c74-30bb-b43d-95dcedf961de',
    'name': 'Colombia',
    'sort-name': 'Colombia',
    'life-span': {'ended': None}},
   'begin-area': {'id': 'c75b683f-3238-4fc8-b5ca-1883ab911561',
    'type': 'City',
    'type-id': '6fd8f29a-3d0a-32fc-980d-ea697b69da78',
    'name': 'Barranquilla',
    'sort-name': 'Barranquilla',
    'life-span': {'ended': None}},
   'disambiguation': 'Colombian singer',
   'ipis': ['00353347465', '00353347661'],
   'isnis': ['0000000116252147', '0000000132386436'],
   'life-span': {'begin': '1977-02-

In [25]:
result.keys()

dict_keys(['created', 'count', 'offset', 'artists'])

In [18]:
result["count"]

968

In [26]:
# Podemos dar por supuesto que es el primer elemento que nos de o iterar entre los dict que tiene la lista resultado y que nos de el valor mas exacto en el name
result["artists"][0]["type"]

'Person'

In [8]:
# Pais origen
result["artists"][0]["area"]["name"]



'Colombia'

In [9]:
# Area origen
result["artists"][0]["begin-area"]["name"]


'Barranquilla'

In [20]:
# Fecha inicio de actividad ()

result["artists"][0]["life-span"]["begin"]

'1990'

In [21]:
# Fecha fin de actividad

result["artists"][0]["life-span"]["ended"]

In [12]:


for i in result["artists"][0]: 
    print(i)

id
type
type-id
score
gender-id
name
sort-name
gender
country
area
begin-area
disambiguation
ipis
isnis
life-span
aliases
tags
